In [1]:
# import libraries
import os
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# model tools
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

# model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
base_dir = '/Users/carlostavarez/Desktop/severstal-steel-defect-detection'

In [3]:
images_tr_dir = base_dir + '/train_images'

In [4]:
df = pd.read_csv(base_dir + '/train.csv')

In [5]:
df.shape

(7095, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7095 entries, 0 to 7094
Data columns (total 3 columns):
ImageId          7095 non-null object
ClassId          7095 non-null int64
EncodedPixels    7095 non-null object
dtypes: int64(1), object(2)
memory usage: 166.4+ KB


In [7]:
df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [8]:
def get_imgs(df):
    
    labels_list = []
    images_list = []
    
    for idx, row in df.iterrows():
        
        img_path = images_tr_dir + '/' + row['ImageId']
        label = row['ClassId']
        
        img = image.load_img(img_path, target_size=(224, 224))
        img = image.img_to_array(img)
        
        labels_list.append(label)
        images_list.append(img)
        
    return np.stack(images_list, axis=0), labels_list

In [9]:
images_list, labels_list = get_imgs(df)

In [10]:
images_list.shape

(7095, 224, 224, 3)

In [11]:
len(images_list)

7095

In [27]:
images_list.shape

(7095, 224, 224, 3)

In [28]:
images_list = images_list.reshape(7095, 224*224*3)

In [29]:
images_list.shape

(7095, 150528)

In [34]:
df_lb = pd.DataFrame(labels_list, columns=['target'])

In [35]:
df_img = pd.DataFrame(images_list)

In [36]:
df_img.shape

(7095, 150528)

In [45]:
df_lb.reset_index()

,index,target
0,0,1
1,1,3
2,2,1
3,3,4
4,4,3
...,...,...
7090,7090,3
7091,7091,3
7092,7092,3
7093,7093,3


In [48]:
df_lb.to_csv('train_target.csv', index=False)

In [49]:
df_img.to_csv('train_imgs.csv', index=False)

In [50]:
df_img.shape

(7095, 150528)

In [47]:
# df_lb.reset_index().join(df_img.reset_index(), on='index')

In [12]:
x_train, x_test, y_train, y_test = train_test_split(images_list, labels_list, 
                                                    random_state=42, stratify=labels_list, train_size=0.9)

In [15]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

/Users/carlostavarez/opt/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [16]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [17]:
# resnet.layers = True

bolck_bol = False

for idx in range(len(resnet.layers)):
    
    if resnet.layers[idx].name == 'res5c_branch2b':
        bolck_bol = True
        
    if bolck_bol:
        resnet.layers[idx].trainable = True
        
    else:
        resnet.layers[idx].trainable = False

In [18]:
model = Sequential()

model.add(resnet)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               51380736  
_________________________________________________________________
dense_2 (Dense)              (None, 200)               102600    
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 804       
Total params: 75,071,852
Trainable params: 54,899,692
Non-trainable params: 20,172,160
_________________________________________________________________


In [20]:
datagen = ImageDataGenerator(rescale=1.0/255)

In [21]:
y_train = np.array(y_train) - 1
y_test = np.array(y_test) - 1

In [22]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [23]:
y_train.shape

(6385, 4)

In [24]:
y_train.shape, x_train.shape

((6385, 4), (6385, 224, 224, 3))

In [25]:
train_gen = datagen.flow(x_train, y_train, batch_size=50)
valid_gen = datagen.flow(x_test, y_test, batch_size=50)

In [26]:
history = model.fit_generator(train_gen, 
                              steps_per_epoch=100, 
                              validation_data=valid_gen, 
                              validation_steps=100, 
                              epochs=10)

Epoch 1/10
100/100 [==============================] - 550s 5s/step - loss: 0.7393 - acc: 0.7661 - val_loss: 0.6927 - val_acc: 0.7256
Epoch 2/10
100/100 [==============================] - 546s 5s/step - loss: 0.3197 - acc: 0.8654 - val_loss: 0.8153 - val_acc: 0.7269
Epoch 3/10
  8/100 [=>............................] - ETA: 4:20 - loss: 0.2985 - acc: 0.8900

KeyboardInterrupt: 